<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Generators/Upload_Processor_DIRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re

client = OpenAI(api_key="")

In [ ]:
df_to_use = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/dailymed_full_text_final.csv")

In [ ]:
df_to_use

In [ ]:
#USE ME

drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
moiety_list = []
fda_label_list = []
set_list = []

for index, row in tqdm(df_to_use.iterrows(), total=df_to_use.shape[0]):




    initial_prompt = """Provide a summary of all the parts of the drug label that discuss kidney toxicity, kidney injury, renal toxicity, and nephrotoxicity risks and kidney toxicity reactions for this drug.
    In your summary of each sentence, clearly state whether the drug itself was associated with or caused the renal toxicity risk."""


    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Renal Toxicity, Less Renal Toxicity, or Most Renal Toxicity?'
        Now, answer with just one word: No, Less or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Renal Toxicity?'
        Now, answer with just one word: Yes or No"""
    initial_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Read through the following drug label"},
        {"role": "user", "content":row["full_text"]},
        {"role": "user", "content":initial_prompt}
        ],
        stream=False, temperature=0,max_tokens=2000
        )
    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["generic_name"])
    moiety_list.append(row["moiety"])
    drugs_initial_responses.append(initial_response.choices[0].message.content.strip())
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    initial_prompts.append(initial_prompt)
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    set_list.append(row["set_id"])
    fda_label_list.append(row["urls"])
    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DIRIL_direct_4o_renaltoxicity_temp_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"set_id":set_list,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DIRIL_direct_4o_renaltoxicity_final.csv")

In [ ]:
print(final_df["less_determination"].value_counts())